In [1]:
import re
from textblob import TextBlob
from tqdm import tqdm
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

c:\Users\zaree\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\zaree\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\random.py:42: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  return default_generator.manual_seed(seed)


In [3]:
DATASET_COLUMNS=['ids','name','location','user_desc','user_created','follow','friends','likes','verified','date','text','hashtag','source','retweets','favourites','is_retweeted']
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('Datasets\covid_vaccine_tweets.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)
df.sample(5)

,ids,name,location,user_desc,user_created,follow,friends,likes,verified,date,text,hashtag,source,retweets,favourites,is_retweeted
164097.0,1425290574710542339,Khondaker Gulam Maula,Bangladesh,I am a professional Lead Generation and Virtua...,2010-11-30 15:55:38,1119,4792,3640,False,2021-08-11 02:58:48,Pls check out https://t.co/bAKiY2zvdU\n#Suarez...,"['Suarez', 'Messi30', 'Coinbase', 'Moderna', '...",Twitter Web App,5,4,False
21582.0,1367654887127883777,dr anil mehndiratta,"Goa, India","Doctor, President #IndianMedicalAssociation #G...",2020-06-14 15:56:47,318,90,7462,False,2021-03-05 01:55:09,An advice 2 d people who hav already recd #cor...,['coronavaccine'],Twitter for iPhone,4,14,False
92312.0,1402160068968255488,EFFORTLESS NEWS Ã°ÂÂÂÃ¯Â¸Â,NaN,Making News Reading EffortlessÃ°ÂÂÂÃ¯Â¸Â\n...,2021-05-26 06:57:49,2,16,1,False,2021-06-08 07:06:25,#Covaxin-Covishield Antibody Study Triggers A ...,"['Covaxin', 'BreakingNews', 'News', 'Effortles...",IFTTT,0,0,False
200743.0,1444344291313082370,AmbetRN,"Los Angeles, CA",BSN RN Ã°ÂÂÂÃ°ÂÂÂºÃ°ÂÂÂ¸Ã°ÂÂÂµÃ°ÂÂÂ...,2016-10-10 03:36:46,21,119,1083,False,2021-10-02 16:51:28,@WHO @WHO_Europe @WHOSEARO @WHOEMRO @WHOWPRO @...,['COVAXIN'],Twitter for iPhone,0,1,False
2750.0,1347266921163608065,Modena,Switzerland,"Individual investor, member of a well informed...",2012-04-04 15:24:12,189,68,3951,False,2021-01-07 19:40:39,91 year old vaccinated dies after few days #Pf...,"['PfizerVaccine', 'PfizerBioNTech']",Twitter for Android,0,0,False


False